In [1]:
import pandas as pd
import numpy as np
from scripts.datacleaningutils import datacleaning
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\maxwe\AppData\Local\Temp\ipykernel_5144\691236561.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Welcome to our data cleaning portion. In this notebook we will
1. Find out if there are missing data
2. Explain how we clean our missing data (the different imputation methods)
### EDA will be done in a different notebook
### Let's begin by importing all our data, we will take all data into account

In [2]:
### For testing purposes, we will take a look at 1 dataframe as reference, before proceeding
df_test = pd.read_csv("data\PRSA_Data_Dongsi_20130301-20170228.csv")
df_test

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Dongsi
1,2,2013,3,1,1,4.0,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,NW,3.9,Dongsi
2,3,2013,3,1,2,7.0,7.0,NaN,17.0,300.0,60.0,-1.2,1025.3,-24.6,0.0,NNW,5.3,Dongsi
3,4,2013,3,1,3,3.0,3.0,5.0,18.0,NaN,NaN,-1.4,1026.2,-25.5,0.0,N,4.9,Dongsi
4,5,2013,3,1,4,3.0,3.0,7.0,NaN,200.0,84.0,-1.9,1027.1,-24.5,0.0,NNW,3.2,Dongsi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,16.0,51.0,3.0,29.0,400.0,73.0,12.5,1013.5,-16.2,0.0,NW,2.4,Dongsi
35060,35061,2017,2,28,20,18.0,45.0,3.0,43.0,500.0,54.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Dongsi
35061,35062,2017,2,28,21,23.0,58.0,5.0,61.0,700.0,28.0,10.8,1014.2,-13.3,0.0,NW,1.1,Dongsi
35062,35063,2017,2,28,22,23.0,53.0,9.0,75.0,900.0,15.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Dongsi


In [4]:
import os
files = os.listdir("data")
files

['PRSA_Data_Aotizhongxin_20130301-20170228.csv',
 'PRSA_Data_Changping_20130301-20170228.csv',
 'PRSA_Data_Dingling_20130301-20170228.csv',
 'PRSA_Data_Dongsi_20130301-20170228.csv',
 'PRSA_Data_Guanyuan_20130301-20170228.csv',
 'PRSA_Data_Gucheng_20130301-20170228.csv',
 'PRSA_Data_Huairou_20130301-20170228.csv',
 'PRSA_Data_Nongzhanguan_20130301-20170228.csv',
 'PRSA_Data_Shunyi_20130301-20170228.csv',
 'PRSA_Data_Tiantan_20130301-20170228.csv',
 'PRSA_Data_Wanliu_20130301-20170228.csv',
 'PRSA_Data_Wanshouxigong_20130301-20170228.csv']

### We then create an array of data frames to store all the data. This will be useful later

In [5]:
df_arr = []
for i in files:
    df_arr.append(pd.read_csv(f"data/{i}"))

In [6]:
print(f"Number of dataframes: {len(df_arr)}\n")
print(f"Number of rows :")
for i in range(len(files)):
    print(f"{files[i]} : {len(df_arr[i])}")
print("")
print(f"Number of missing Data :")
for i in range(len(files)):
    print(f"{files[i]} : {(df_arr[i].isnull().sum().max())}")
    # print("=================================")

Number of dataframes: 12

Number of rows :
PRSA_Data_Aotizhongxin_20130301-20170228.csv : 35064
PRSA_Data_Changping_20130301-20170228.csv : 35064
PRSA_Data_Dingling_20130301-20170228.csv : 35064
PRSA_Data_Dongsi_20130301-20170228.csv : 35064
PRSA_Data_Guanyuan_20130301-20170228.csv : 35064
PRSA_Data_Gucheng_20130301-20170228.csv : 35064
PRSA_Data_Huairou_20130301-20170228.csv : 35064
PRSA_Data_Nongzhanguan_20130301-20170228.csv : 35064
PRSA_Data_Shunyi_20130301-20170228.csv : 35064
PRSA_Data_Tiantan_20130301-20170228.csv : 35064
PRSA_Data_Wanliu_20130301-20170228.csv : 35064
PRSA_Data_Wanshouxigong_20130301-20170228.csv : 35064

Number of missing Data :
PRSA_Data_Aotizhongxin_20130301-20170228.csv : 1776
PRSA_Data_Changping_20130301-20170228.csv : 1521
PRSA_Data_Dingling_20130301-20170228.csv : 2012
PRSA_Data_Dongsi_20130301-20170228.csv : 3197
PRSA_Data_Guanyuan_20130301-20170228.csv : 1753
PRSA_Data_Gucheng_20130301-20170228.csv : 1401
PRSA_Data_Huairou_20130301-20170228.csv : 1639
P

# ---------------------------------------------------------------------------------------------------------
### We need to impute the data somehow. We can check for the best locations to impute missing data.
### Idea : find district in Beijing 7km away from each in our dataset, and check correlation between columns.
### Reason : Correlation != Causation. It doesn't make sense for a district far away from another to have the same weather conditions. It could just be pure coincidence

### Step 1: Plot out each district on google maps
<img src="img/gmap1.png" alt="gmap" height="300"></img>
### Step 2: Identify closest all datasets within 7km of each other (using google measure tool)
<img src="img/gmap2.png" alt="gmap" height="300"></img>
### We have found Dongsi, Guanyuan, Nongzhanguan and Tiantan to be the closest 4 cities.

In [7]:
names = ['PRSA_Data_Dongsi_20130301-20170228.csv',
 'PRSA_Data_Guanyuan_20130301-20170228.csv',
 'PRSA_Data_Nongzhanguan_20130301-20170228.csv',
 'PRSA_Data_Tiantan_20130301-20170228.csv']

### We have written a library to make the data cleaning process easier. We will briefly explain the use of each function, and attach the source code in datacleaningutils.py

In [8]:
# import all data
df_arr = []
for i in names:
    df_arr.append(pd.read_csv(f"data/{i}"))

### The cell below returns the number of rows that contains missing data

In [9]:
print(f"Number of dataframes: {len(df_arr)}\n")
print(f"Number of rows :")
for i in range(len(names)):
    print(f"{names[i]} : {len(df_arr[i])}")
print("")
print(f"Number of names Data :")
for i in range(len(names)):
    print(f"{names[i]} : {(df_arr[i].isnull().sum().max())}")
    # print("=================================")

Number of dataframes: 4

Number of rows :
PRSA_Data_Dongsi_20130301-20170228.csv : 35064
PRSA_Data_Guanyuan_20130301-20170228.csv : 35064
PRSA_Data_Nongzhanguan_20130301-20170228.csv : 35064
PRSA_Data_Tiantan_20130301-20170228.csv : 35064

Number of names Data :
PRSA_Data_Dongsi_20130301-20170228.csv : 3197
PRSA_Data_Guanyuan_20130301-20170228.csv : 1753
PRSA_Data_Nongzhanguan_20130301-20170228.csv : 1206
PRSA_Data_Tiantan_20130301-20170228.csv : 1126


### On the other hand, the cell below uses our custom function to identify windows of missing data for each column

In [10]:
from scripts.datacleaningutils import datacleaning
cleaner = datacleaning()

In [11]:
for i in range(len(names)):
    print(names[i])
    for n in df_arr[i].columns:
        print(n)
        cleaner.find_missing(df_arr[i], column = n)
    print("=======================")

PRSA_Data_Dongsi_20130301-20170228.csv
No
year
month
day
hour
PM2.5
Windows of missing data:
Number of windows of missing data 214
longest window of missing data: 214
Point missing data 113
PM10
Windows of missing data:
Number of windows of missing data 179
longest window of missing data: 179
Point missing data 115
SO2
Windows of missing data:
Number of windows of missing data 204
longest window of missing data: 204
Point missing data 123
NO2
Windows of missing data:
Number of windows of missing data 204
longest window of missing data: 204
Point missing data 129
CO
Windows of missing data:
Number of windows of missing data 304
longest window of missing data: 304
Point missing data 199
O3
Windows of missing data:
Number of windows of missing data 247
longest window of missing data: 247
Point missing data 164
TEMP
Windows of missing data:
Number of windows of missing data 8
longest window of missing data: 8
Point missing data 5
PRES
Windows of missing data:
Number of windows of missing d

### Yikes, looks like we've got tons of missing data in different columns
### Following this, we have written a function to clean the data using

1. **Combining data from highly correlated data sets**
2. **Linear interpolation (only for POINT missing data; i.e [1,NULL,1] -> [1,1,1], [1,NULL,NULL,1] is not suitable)**
3. **As a last resort, KNN imputation**

Feel free to take a look a the source code to see exactly how we do it in **scripts/datacleaningutils.py**

Note that our cleaner only can impute Numerical, Continuous variables.

The Numeric Variables are ['year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']

Although year, month, day, hour are technically categories, there exists no missing data within those columns, hence they will not be affected.

We also decided to ignore the station as we are going to create a combined dataset in the heart of beijing.

The only categorical column we are going to look at is wd (wind direction)

In [13]:
df_arr_numeric_only = []
for i in names:
    df_arr_numeric_only.append(pd.read_csv(f"data/{i}")[['year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']])

In [14]:
df_arr_clean = cleaner.clean_data(df_arr_numeric_only) # this will clean the data

In [15]:
for i in range(len(names)):
    print(names[i])
    for n in df_arr_clean[i].columns:
        print(n)
        cleaner.find_missing(df_arr_clean[i], column = n)
    print("=======================")

PRSA_Data_Dongsi_20130301-20170228.csv
year
month
day
hour
PM2.5
PM10
SO2
NO2
CO
O3
TEMP
PRES
DEWP
RAIN
WSPM
PRSA_Data_Guanyuan_20130301-20170228.csv
year
month
day
hour
PM2.5
PM10
SO2
NO2
CO
O3
TEMP
PRES
DEWP
RAIN
WSPM
PRSA_Data_Nongzhanguan_20130301-20170228.csv
year
month
day
hour
PM2.5
PM10
SO2
NO2
CO
O3
TEMP
PRES
DEWP
RAIN
WSPM
PRSA_Data_Tiantan_20130301-20170228.csv
year
month
day
hour
PM2.5
PM10
SO2
NO2
CO
O3
TEMP
PRES
DEWP
RAIN
WSPM


### Just like that, we have successfully filled in all Numeric values.
So what are we going to do about the Categorical Values?

We are going to fill in the categorical values with the *mode*

We will look at the most common win direction and use majority polling among the df_arr

In [16]:
seen = {}
for i in range(len(names)):
    mode = df_arr[i]['wd'].mode().values[0]
    if mode not in seen:
        seen[mode] = 1
    else:
        seen[mode] = seen[mode] + 1
seen

{'ENE': 3, 'NE': 1}

### Knowing that ENE is the most common wind direction, we fill the missing wind directions into the dataset

In [17]:
print(f"Null values found: {df_arr[0]['wd'].isnull().sum()}")
wd = df_arr[0]['wd'].fillna("ENE")
print(f"After filling: {wd.isnull().sum()}")

Null values found: 78
After filling: 0


### Following this, we also merge all dataframes into the a single dataframe by taking the mean value between all numeric columns. This is one through our datacleaner library

In [18]:
res = cleaner.merge_dataframes_mean(df_arr_numeric_only)

### Almost there... Now we just need to add in the missing wd (wind direction column)

In [19]:
res['wd'] = wd

In [20]:
res

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM,wd
0,2013.0,3.0,1.0,0.0,6.00,8.25,6.25,14.25,275.0,81.00,-0.550,1024.125,-20.750,0.0,5.375,NNW
1,2013.0,3.0,1.0,1.0,5.50,12.25,6.75,14.00,275.0,81.00,-0.800,1024.625,-21.125,0.0,4.100,NW
2,2013.0,3.0,1.0,2.0,4.75,5.50,7.25,15.50,275.0,71.75,-1.175,1024.850,-23.000,0.0,5.375,NNW
3,2013.0,3.0,1.0,3.0,4.25,5.00,5.25,17.00,325.0,70.50,-1.400,1025.775,-23.975,0.0,4.450,N
4,2013.0,3.0,1.0,4.0,4.00,4.75,6.25,16.00,300.0,75.50,-1.925,1026.625,-23.250,0.0,2.900,NNW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2017.0,2.0,28.0,19.0,15.75,39.25,2.75,30.25,425.0,69.50,12.500,1013.500,-16.200,0.0,2.400,NW
35060,2017.0,2.0,28.0,20.0,16.75,37.25,3.25,41.00,475.0,52.50,11.600,1013.600,-15.100,0.0,0.900,WNW
35061,2017.0,2.0,28.0,21.0,18.00,40.50,4.75,49.50,575.0,41.75,10.800,1014.200,-13.300,0.0,1.100,NW
35062,2017.0,2.0,28.0,22.0,15.00,38.50,6.25,55.75,625.0,35.25,10.500,1014.400,-12.900,0.0,1.200,NNW


### Et Voila! We finally have our data cleaned! Next step is to export the data as a CSV

In [22]:
res.to_csv("data/cleaned.csv", index=False)

### The next step is to explore our data. This will be done in another notebook!